In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_file_path = '/content/drive/MyDrive/Tidy room nss/alphabets_dataset.zip'

extract_folder = '/content/extracted_files'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print('Files extracted to:', extract_folder)

Files extracted to: /content/extracted_files


In [ ]:
import shutil
csv_file_path = '/content/drive/MyDrive/Tidy room nss/target_labels.csv'

destination_path = '/content/target_labels.csv'
shutil.copy(csv_file_path, destination_path)

'/content/target_labels.csv'

In [ ]:
import shutil
csv_file_path = '/content/drive/MyDrive/Tidy room nss/sentiment_analysis_dataset.csv'

destination_path = '/content/sentiment_analysis_dataset.csv'
shutil.copy(csv_file_path, destination_path)

'/content/sentiment_analysis_dataset.csv'

In [ ]:
import os
images_path = '/content/drive/MyDrive/Tidy room nss/target_images'
os.listdir(images_path)

import shutil
local_images_path = '/content/images_folder'
shutil.copytree(images_path, local_images_path)
os.listdir(local_images_path)


['line_5.png',
 'line_2.png',
 'line_1.png',
 'line_4.png',
 'line_3.png',
 'line_6.png']

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

images_path = '/content/extracted_files/alphabet_images'
labels_file = '/content/extracted_files/alphabet_labels.csv'

labels_df = pd.read_csv(labels_file)
print(labels_df.head())
print(labels_df.shape)
image_files = [f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

print(f"Number of image files: {len(image_files)}")

image_size = (28, 28)

def process_image(filename, label):
    img_path = os.path.join(images_path, filename)
    img = tf.keras.preprocessing.image.load_img(img_path, color_mode='grayscale', target_size=image_size)
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize pixel values
    return img, label

def load_and_preprocess_from_path_label(row):
    filename = row['file']
    label = row['label']
    label = labels_df[label]
    return process_image(filename, label)

def encode_label(label):
    return tf.keras.utils.to_categorical(label, num_classes=len(labels_df))


          file label
0  image_1.png     A
1  image_2.png     A
2  image_3.png     A
3  image_4.png     A
4  image_5.png     A
(372451, 2)
Number of image files: 372451


In [ ]:
file_path = '/content/drive/MyDrive/Tidy room nss/alphabet_labels.csv'

with open(file_path, 'r') as file:
    lines = file.readlines()
    last_5_lines = lines[-5:]

    for line in last_5_lines:
        print(line.strip())  # .strip() to remove extra newline characters

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

images_path = '/content/extracted_files/alphabet_images'
labels_file = '/content/extracted_files/alphabet_labels.csv'

labels_df = pd.read_csv(labels_file)
print(labels_df.head())
print(labels_df.shape)

image_files = [f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
print(f"Number of image files: {len(image_files)}")

image_size = (28, 28)

def load_images_and_labels():
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        filename = row['file']
        label = row['label']
        img_path = os.path.join(images_path, filename)
        if os.path.exists(img_path):
            img = tf.keras.preprocessing.image.load_img(img_path, color_mode='grayscale', target_size=image_size)
            img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            images.append(img)
            labels.append(label)
        else:
            print(f"Image {img_path} not found.")
    return np.array(images), np.array(labels)

images, labels = load_images_and_labels()

unique_labels = sorted(set(labels))
label_to_index = {label: index for index, label in enumerate(unique_labels)}
indexed_labels = np.array([label_to_index[label] for label in labels])

from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(images, indexed_labels, test_size=0.2, random_state=42)

print(f"Total dataset size: {len(images)}")
print(f"Training dataset size: {len(train_images)}")
print(f"Validation dataset size: {len(val_images)}")

from tensorflow.keras import layers, models

num_classes = len(unique_labels)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))


          file label
0  image_1.png     A
1  image_2.png     A
2  image_3.png     A
3  image_4.png     A
4  image_5.png     A
(372451, 2)
Number of image files: 372451
Total dataset size: 372451
Training dataset size: 297960
Validation dataset size: 74491
Epoch 1/10
9312/9312 [==============================] - 61s 6ms/step - loss: 0.1353 - accuracy: 0.9616 - val_loss: 0.0672 - val_accuracy: 0.9808
Epoch 2/10
9312/9312 [==============================] - 51s 5ms/step - loss: 0.0542 - accuracy: 0.9845 - val_loss: 0.0445 - val_accuracy: 0.9873
Epoch 3/10
9312/9312 [==============================] - 51s 6ms/step - loss: 0.0402 - accuracy: 0.9883 - val_loss: 0.0377 - val_accuracy: 0.9892
Epoch 4/10
9312/9312 [==============================] - 51s 5ms/step - loss: 0.0313 - accuracy: 0.9906 - val_loss: 0.0358 - val_accuracy: 0.9901
Epoch 5/10
9312/9312 [==============================] - 56s 6ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.0410 - val_accuracy: 0.9888
Epoch 6/10
9312/931

In [ ]:
test_images_path= '/content/images_folder'

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

test_images = []
test_image_files = [f for f in os.listdir(test_images_path) if os.path.isfile(os.path.join(test_images_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

for file in test_image_files:
    img_path = os.path.join(test_images_path, file)
    img = load_img(img_path, color_mode='grayscale', target_size=(28, 28))
    img = img_to_array(img) / 255.0
    test_images.append(img)

test_images = np.array(test_images)

In [ ]:
predicted_labels = model.predict(test_images)
predicted_labels_indices = np.argmax(predicted_labels, axis=1)


1/1 [==============================] - 0s 243ms/step


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

def predict_letter(image, model, image_size=(28, 28)):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, image_size)
    # Check if the image is mostly blank
    if np.mean(img)==0.0:
        return ' '
    img = img.reshape((1, image_size[0], image_size[1], 1)) / 255.0
    prediction = model.predict(img)
    predicted_letter = chr(np.argmax(prediction) + 65)
    return predicted_letter

def segment_image(image_path, segment_width=28, segment_height=28):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    segments = []

    for y in range(0, height, segment_height):
        for x in range(0, width, segment_width):
            segment = image[y:y + segment_height, x:x + segment_width]
            if segment.shape[0] == segment_height and segment.shape[1] == segment_width:
                segments.append(segment)

    return segments

def predict_sentence(image_path, model):
    segments = segment_image(image_path)
    sentence = ''.join([predict_letter(segment, model) for segment in segments])
    return sentence

test_image_path = '/content/images_folder/line_2.png'

predicted_sentence = predict_sentence(test_image_path, model)
print("Predicted Sentence:", predicted_sentence)


1/1 [==============================] - 0s 18ms/step
Predicted Sentence: IT IS FRUSTRATING THAT YOU NEVER PAY ATTENTION DURING DISCUSSIONS AND YOUR LACK OF FOCUS IS REALLY AFFECTING OUR PROGRESS         


In [ ]:
import pandas as pd

sentiment_analysis_path = '/content/sentiment_analysis_dataset.csv'
sentiment_df = pd.read_csv(sentiment_analysis_path)

sentiment_df['line'] = sentiment_df['line'].apply(lambda x: x.lower())
sentiment_df['line'] = sentiment_df['line'].str.replace('[^\w\s]', '')

print(sentiment_df.head())


                                                line sentiment
0  i am really frustrated because you constantly ...     Angry
1  it makes me upset that you never take responsi...     Angry
2  i cannot believe you missed another deadline a...     Angry
3  it annoys me when you interrupt during meeting...     Angry
4  i am tired of your excuses every time somethin...     Angry


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

sentiment_analysis_path = '/content/sentiment_analysis_dataset.csv'
sentiment_df = pd.read_csv(sentiment_analysis_path)

def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^\w\s]', '', text)
    return text

sentiment_df['line'] = sentiment_df['line'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(sentiment_df['line'], sentiment_df['sentiment'], test_size=0.2, random_state=42)

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(X_test)
print(y_test)
print(y_pred)
print(f"Sentiment analysis model accuracy: {accuracy * 100:.2f}%")


27    your email was clear and informative addressin...
15    it makes me happy that you always recognize th...
23    the event yesterday was wellorganized and the ...
17    it is amazing how you always stay calm under p...
8     i cannot stand how you always talk down to peo...
9     it is very disappointing that you never follow...
Name: line, dtype: object
27    Neutral
15      Happy
23    Neutral
17      Happy
8       Angry
9       Angry
Name: sentiment, dtype: object
['Neutral' 'Angry' 'Neutral' 'Happy' 'Happy' 'Angry']
Sentiment analysis model accuracy: 66.67%


In [ ]:
def predict_sentiment(text, model):
    text = preprocess_text(text)
    return model.predict([text])[0]

target_sentiment_labels_path = '/content/target_labels.csv'
target_sentiment_labels_df = pd.read_csv(target_sentiment_labels_path)

extracted_texts = [
    predict_sentence('/content/images_folder/line_1.png', model),
    predict_sentence('/content/images_folder/line_2.png', model),
    predict_sentence('/content/images_folder/line_3.png', model),
    predict_sentence('/content/images_folder/line_4.png', model),
    predict_sentence('/content/images_folder/line_5.png', model),
    predict_sentence('/content/images_folder/line_6.png', model),
]

predicted_sentiments = [predict_sentiment(text, pipeline) for text in extracted_texts]

target_sentiments = target_sentiment_labels_df['sentiment'].tolist()[:len(predicted_sentiments)]
print(predicted_sentiments)
sentiment_accuracy = accuracy_score(target_sentiments, predicted_sentiments)
print(f"Sentiment Analysis Accuracy: {sentiment_accuracy * 100:.2f}%")


1/1 [==============================] - 0s 26ms/step
['Happy', 'Angry', 'Happy', 'Happy', 'Neutral', 'Neutral']
Sentiment Analysis Accuracy: 83.33%
